In [24]:
#Bokeh plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Span, Label, Arrow, NormalHead
from bokeh.models import HoverTool, tools, ColumnDataSource, CustomJS, Slider, BoxAnnotation
from bokeh.layouts import  column, row
from bokeh.palettes import Category20_18
import re
import glob, os
from astropy.io import fits
from urllib import urlretrieve
from pyraf import iraf
output_notebook()
iraf.noao.onedspec()
iraf.dataio()

Loading BokehJS ...

In [91]:
#Emission Lines
class line(object):
    def __init__(self,name,linecenter,regiontofit):
        self.name = name
        self.linecenter = linecenter
        self.regiontofit = regiontofit
        
#Parametest for the fit:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx, array[idx]

def isDigit(x):
    try:
        float(x)
        return True
    except ValueError:
        return False
    
def gaussian(x, mu, sig,core):
    return core*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

#Emission Lines
class line(object):
    def __init__(self,name,linecenter,regiontofit):
        self.name = name
        self.linecenter = linecenter
        self.regiontofit = regiontofit
        

class spec(object):
    def __init__(self,url):
        self.url = url
        self.name = url.split('/')[-1]
        self.wave, self.flux, self.model, self.type = self.get_type()

        
    def get_fit(self):
        if not os.path.isfile(self.name.strip()):
            urlretrieve(self.url,name.strip())
            
            
    def get_type(self):
        if not os.path.exists(self.name):
            self.get_fit()
        ob = fits.open(self.name)
            #Get data and save wavelenft in x, and flux on y
        dataob = ob[1].data
        x = 10**(dataob['loglam'])
        y2 = dataob['model']
        y = dataob['flux']
        typee = ob[2].data['SUBCLASS'][0]
        return x,y,y2, typee
    
    def get_iraffits(self, modelfit = True):
        if modelfit:
            fluxormodel = self.model
        else:
            fluxormodel = self.flux
        namenewfits = '{name}.txt'.format(name=self.name)
        with open(namenewfits,'w') as file:
            for x,y in zip(self.wave,fluxormodel):
                file.write('{}\t{}\n'.format(x,y))
        #Create fits file from text file interpolation to work wtih IRAF functions
        iraffitsname = 'iraf'+self.name
        if os.path.exists(iraffitsname+'.fits'):
            os.remove(iraffitsname+'.fits')
        iraf.rspectext(namenewfits,iraffitsname,dtype='interp')
        os.remove(namenewfits)
        
    def fit_lines(self, dicoflines, errorestimate = True):
        self.linesdicall = []
        errorparam = []
        if not os.path.exists('iraf'+self.name):
            self.get_iraffits()
            
        #Initialize files log and lines
        ! echo '' > fited.log

        for indexline,linesfit in enumerate(dicoflines):
            regionf = "{} {}".format(linesfit.regiontofit[0],linesfit.regiontofit[1])
            #wavelenght
            xlimns = [find_nearest(self.wave,i)[0] for i in linesfit.regiontofit   ]
            wavex = self.wave[xlimns[0]:xlimns[1]]
            lineszero = linesfit.linecenter
            ! echo '$lineszero' > lines.lines

            filename = 'iraf'+self.name
            #Error estimation iraf: http://stsdas.stsci.edu/cgi-bin/gethelp.cgi?fitprofs
            if errorestimate:        
                iraf.fitprofs(filename,pos='lines.lines', reg=regionf ,
                              fitbackground= 'yes', 
                              logfile='fited.log'
                              ,nerrsample='100',sigma0='4',invgain='4')
            else:
                iraf.fitprofs(filename,pos='lines.lines', reg=regionf ,
                              fitbackground= 'yes', 
                              logfile='fited.log')

            #Plotting the gaussian
            #Find in log file of fitprofs.  
            npattern = re.compile('[-\d.]+')
            npattern2 = re.compile('[-\d.E?]+') #Gets the exponentioals

            gparameters=[]
            with open('fited.log','r') as file:
                for lines in file:

                    if '(' not in lines:
                        temp = npattern2.findall(lines)
                        if 'INDEF' in lines:
                            gparameters.append(7*[0])
                    else:
                        errorparam.append(npattern2.findall(lines))

                    if len(temp) == 7 and all(isDigit(i) for i in temp) and 'INDEF' not in lines:
                        gparameters.append(temp)

            #gaussian
            if len(gparameters) > 0:
                gparamfinal = [ float(i) for i in gparameters[-1] ]
                centerg, contg, fluxg, eqwg, coreg, fwhmg, fwhml = gparamfinal
                yg = gaussian(wavex,centerg,fwhmg/2.3538,coreg) + contg


            if errorestimate == True:
                errorparamfinal = [ float(i) for i in errorparam[-1] ]
                print(gparamfinal)
                print(errorparamfinal)


                linesdic = {'linename':linesfit.name,
                             'center':centerg,
                              'EW': eqwg,
                              'EWerror':errorparamfinal[3],
                            'fluxg':fluxg,
                            'coreg':coreg,
                            'fwgmg':fwhmg,
                            'contg':contg,
                            'gaussian':{'x':wavex,'y':yg}
                            }
                self.linesdicall.append(linesdic)

        
        


In [92]:
tryspec = spec('https://dr14.sdss.org/sas/dr14/sdss/spectro/redux/26/spectra/0266/spec-0266-51630-0015.fits')

In [93]:
tryspec.type

'A0'

In [94]:
diclines = {line('P15',8547,[8520,8560]),line('P14',8600,[8600-20,8600+20]),
           line('Halpha',6562,[6550,6575]),line('P16',8504,[8504-20,8504+20]),
            line('P13',8667,[8667-20,8667+20]),line('P12',8752,[8752-20,8752+20]),
           line('P11',8865,[8865-20,8865+20]),line('P10',9017,[9017-20,9017+20])}


tryspec.fit_lines(diclines)

# Feb  2 16:27 irafspec-0266-51630-0015.fits - Ap 1: 
# Nfit=1, background=yes, positions=all, gfwhm=all, lfwhm=all
#   center      cont      flux       eqw      core     gfwhm     lfwhm
  8757.704  9.671695  -15.1684     1.568  -1.04753      13.6        0.
 (42.7734) (3.00002) ( 1046.2) (  13.02) (25.9032) (  54.83) (     0.)
[8757.704, 9.671695, -15.1684, 1.568, -1.04753, 13.6, 0.0]
[42.7734, 3.00002, 1046.2, 13.02, 25.9032, 54.83, 0.0]
# Feb  2 16:27 irafspec-0266-51630-0015.fits - Ap 1: 
# Nfit=1, background=yes, positions=all, gfwhm=all, lfwhm=all
#   center      cont      flux       eqw      core     gfwhm     lfwhm
  8670.916  9.719036  -9.31016    0.9579  -1.18503     7.381        0.
 (65.1855) (2.20775) (155.929) (  8.732) (12.9799) (   20.6) (     0.)
[8670.916, 9.719036, -9.31016, 0.9579, -1.18503, 7.381, 0.0]
[65.1855, 2.20775, 155.929, 8.732, 12.9799, 20.6, 0.0]
# Feb  2 16:27 irafspec-0266-51630-0015.fits - Ap 1: 
# Nfit=1, background=yes, positions=all, gfwhm=all, lfwhm=

In [107]:
tryspec.linesdicall[0]['gaussian']['x']

array([ 8731.72167969,  8733.73535156,  8735.74414062,  8737.75878906,
        8739.76855469,  8741.78417969,  8743.79492188,  8745.81152344,
        8747.82324219,  8749.8359375 ,  8751.85351562,  8753.8671875 ,
        8755.88574219,  8757.90039062,  8759.91992188,  8761.93457031,
        8763.95019531,  8765.97167969,  8767.98828125,  8770.00976562], dtype=float32)

In [119]:
#plot parameters
colors = 100*Category20_18 #+ Viridis8 + Dark2_8 + Paired8 + Set1_4
xr = (5000,8900)
yr = (0,40)
pl =  figure(x_axis_label='Angstrom', y_axis_label='Y',title="Click on the desired stellar template to overplot", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=1000
             )

hover2 = HoverTool(
        tooltips=[
            ("(x,y)", "($x{1}, $y)"),
        ]
    )
pl.add_tools(hover2)

index =0



#If you want to have the name and not the spectra type change legend=name
t = pl.line(x=tryspec.wave,y=tryspec.model,color=colors[index], line_alpha=1.0, 
            line_width=4,legend=tryspec.name,muted_alpha=0.,muted_color=colors[index])


for i in tryspec.linesdicall:
    
    pl.line(i['gaussian']['x'],i['gaussian']['y'],color='red', line_alpha=1.,
            line_width=5,legend=tryspec.name,muted_alpha=0.,muted_color=colors[index])



pl.legend.location = "top_left"
pl.legend.click_policy="mute"

show(pl)  

In [117]:
for i in tryspec.linesdicall:

    print(i['line'])

KeyError: 'line'

# Old way

['../listA0.txt',
 '../listA0index:1.txt',
 '../listA0IVn.txt',
 '../listA0p.txt',
 '../listA1III.txt',
 '../listA1Iae.txt',
 '../listA1V.txt',
 '../listA1m.txt',
 '../listA2II.txt',
 '../listA2Ia.txt',
 '../listA3Iae.txt',
 '../listA4.txt',
 '../listA4V.txt',
 '../listA4Vindex:13.txt',
 '../listA4p.txt',
 '../listA5II.txt',
 '../listA5Ia.txt',
 '../listA6IV.txt',
 '../listA8V.txt',
 '../listA9V.txt',
 '../listAm.txt']

In [22]:
files = glob.glob('../listA*.txt')
for finames in files:
    with open(finames,'r') as f:
        for lines in f:
            star = lines.split(',')[0]
            url= lines.split(',')[1]
            name = url.split('/')[-1]
            if not os.path.isfile(name.strip()):
                urlretrieve(url,name.strip())
        #print(url)

KeyboardInterrupt: 

In [ ]:
ls